# 🍜 Kaggle MCP Agent

https://www.kaggle.com/blog/kaggles-official-mcp-server

# 1. ⚙️ Setup

In [1]:
from utilities.setup import *

In [2]:
GOOGLE_API_KEY = get_google_api_key()
retry_config = get_retry_config()
KAGGLE_API_KEY = get_kaggle_credentials()

✅ Gemini API key setup complete.
✅ Kaggle API key setup complete.


# 2. 🧑‍💻 Define Kaggle MCP Agent

In [3]:
mcp_kaggle_server = McpToolset(
    connection_params=StdioConnectionParams(
        server_params=StdioServerParameters(
            command="npx",
            args=[
                "-y",
                "mcp-remote",
                "https://www.kaggle.com/mcp",
                "--header",
                f"Authorization: Bearer {KAGGLE_API_KEY}"
            ],
            # tool_filter=["search_notebooks", "get_notebook_info"]
        ),
        timeout=60
    )
)

print("✅ MCP Kaggle Toolset defined.")

✅ MCP Kaggle Toolset defined.


In [4]:
mcp_kaggle_server.get_tools("list_active_competitions")

<coroutine object McpToolset.get_tools at 0x10d7f4c40>

In [5]:
kaggle_agent = LlmAgent(
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    name="kaggle_agent",
    instruction="""You are a Kaggle Data Science assistant. Your goal is to assist users with Kaggle competitions, datasets, and notebooks using the provided MCP tools.

    CRITICAL OPERATION RULES:
    1. Notebooks: You typically do not know the exact 'user_name' or 'slug' of a notebook. ALWAYS use 'search_notebooks' first to find the correct author and slug before attempting to use 'get_notebook_info'. Never guess the username.

    2. Competitions: You typically do not know the exact competition 'ref' (slug) or ID. ALWAYS use 'search_competitions' first to identify the correct competition reference before attempting to use tools like 'get_competition_files', 'get_competition_leaderboard', or 'get_competition_details'.

    3. General: If a user asks for a specific item (e.g., "The Titanic Competition"), do not hallucinate the ID. Search for it, confirm the ID from the results, and then fetch the details.""",
    tools=[mcp_kaggle_server]
)

In [6]:
# kaggle_agent = LlmAgent(
#     model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
#     name="kaggle_agent",
#     instruction="""You are a Kaggle Data Science assistant. Use the provided Kaggle MCP tools to search for competitions, find relevant datasets, and retrieve details about notebooks/kernels based on the user’s request.
#
#     CRITICAL RULE: You typically do not know the exact 'user_name' or 'slug' of a notebook. ALWAYS use 'search_notebooks' first to find the correct author and slug before attempting to use 'get_notebook_info'. Never guess the username.""",
#     tools=[mcp_kaggle_server]
# )

In [7]:
runner = InMemoryRunner(agent=kaggle_agent)

In [8]:
response = await runner.run_debug("Could you please provide a concise and structured overview of what the “Testing o1-preview on ARC-AGI” notebook entails?", verbose=True)


 ### Created new session: debug_session_id

User > Could you please provide a concise and structured overview of what the “Testing o1-preview on ARC-AGI” notebook entails?


/Users/myte/Documents/Projects/KaggleN/venv/lib/python3.11/site-packages/google/adk/tools/mcp_tool/mcp_tool.py:103: UserWarning: [EXPERIMENTAL] BaseAuthenticatedTool: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  super().__init__(


kaggle_agent > [Calling tool: search_notebooks({'request': {'search': 'Testing o1-preview on ARC-...)]
kaggle_agent > [Tool result: {'content': [{'type': 'text', 'text': '{\n  "kernels": [\n    {\n      "ref": "gregkamradt/testing-o...]
kaggle_agent > [Calling tool: get_notebook_info({'request': {'userName': 'gregkamradt', 'kernelSlu...)]
kaggle_agent > [Tool result: {'content': [{'type': 'text', 'text': '{\n  "metadata": {\n    "id": 65737915,\n    "ref": "gregkamr...]
kaggle_agent > The notebook "Testing o1-preview on ARC-AGI" by Greg Kamradt is a starter notebook designed to help users test OpenAI models, specifically the "o1-preview" model, on the ARC-AGI (Abstraction and Reasoning Challenge) dataset.

Here's a breakdown of its key aspects:

*   **Purpose:** To provide a basic framework for interacting with OpenAI models for the ARC-AGI challenge.
*   **Functionality:**
    *   It includes code to install necessary libraries (`openai`, `pydantic`).
    *   It demonstrates how to lo

In [9]:
response

[Event(model_version='gemini-2.5-flash-lite', content=Content(
   parts=[
     Part(
       function_call=FunctionCall(
         args={
           'request': {
             'search': 'Testing o1-preview on ARC-AGI',
             'sortBy': 'Relevance'
           }
         },
         id='adk-a352b385-8150-4eb3-a1e9-4ff90924b0ab',
         name='search_notebooks'
       )
     ),
   ],
   role='model'
 ), grounding_metadata=None, partial=None, turn_complete=None, finish_reason=<FinishReason.STOP: 'STOP'>, error_code=None, error_message=None, interrupted=None, custom_metadata=None, usage_metadata=GenerateContentResponseUsageMetadata(
   candidates_token_count=36,
   prompt_token_count=14215,
   prompt_tokens_details=[
     ModalityTokenCount(
       modality=<MediaModality.TEXT: 'TEXT'>,
       token_count=14215
     ),
   ],
   total_token_count=14251
 ), live_session_resumption_update=None, input_transcription=None, output_transcription=None, avg_logprobs=None, logprobs_result=None, ca

In [10]:
response = await runner.run_debug("How many upvotes and comments does the “Testing o1-preview on ARC-AGI” notebook have? user: Greg Kamradt", verbose=True)


 ### Continue session: debug_session_id

User > How many upvotes and comments does the “Testing o1-preview on ARC-AGI” notebook have? user: Greg Kamradt
kaggle_agent > I can only retrieve the total number of upvotes for the notebook, which is 61. I cannot retrieve the number of comments.


🎉 Completed the **🕵🏻‍♂️ Kaggle MCP Agent**.

In [11]:
message = "Who are the top 3 in the leaderboard in the AI Mathematical Olympiad - Progress Prize 3 Competitions?"

response = await runner.run_debug(message, verbose=True)


 ### Continue session: debug_session_id

User > Who are the top 3 in the leaderboard in the AI Mathematical Olympiad - Progress Prize 3 Competitions?
kaggle_agent > I need a more specific competition name to search the leaderboard. Could you please provide the exact name of the competition?


In [12]:
name = "AI Mathematical Olympiad - Progress Prize 3 (Solve international-level math challenges using artificial intelligence models)"
message = f"Who are the top 3 in the leaderboard in the {name} Competitions?"

response = await runner.run_debug(message, verbose=True)


 ### Continue session: debug_session_id

User > Who are the top 3 in the leaderboard in the AI Mathematical Olympiad - Progress Prize 3 (Solve international-level math challenges using artificial intelligence models) Competitions?
kaggle_agent > [Calling tool: search_competitions({'request': {'search': 'AI Mathematical Olympiad -...)]
kaggle_agent > [Tool result: {'content': [{'type': 'text', 'text': '{\n  "competitions": [\n    {\n      "id": 118448,\n      "re...]
kaggle_agent > [Calling tool: get_competition_leaderboard({'request': {'competitionName': 'ai-mathematical-o...)]
kaggle_agent > [Tool result: {'content': [{'type': 'text', 'text': '{\n  "submissions": [\n    {\n      "team_id": 14800490,\n   ...]
kaggle_agent > The top 3 teams on the leaderboard for the "AI Mathematical Olympiad - Progress Prize 3" competition are:

1.  **Yi-Chia Chen** with a score of 38
2.  **Just a test** with a score of 36
3.  **Tong Hui Kang** with a score of 34
